# Construcción de un chatBot

## Preprocesamiento de la información

Para preprocesar el texto voy a utilizar la libreria construida para NLP spaCy. La información que voya a tartar se recoge de una base de datos nutrida utilizando la API REST de Stack Overflow, que recoge preguntas y respuestas sobre Python y Machine learning.

Compruebo la conexión a la bd

In [ ]:
#!pip install 

In [11]:
import sqlite3
import spacy

nlp = spacy.load("en_core_web_sm")

Accedo a la base de datos para tratar la información a un formato que sirva para entrenar una red neuronal. Para tratar el texto se tokeniza por palabras, quedándome unicamente con las caracteres alphanuméricos, lo que dejará fuera espacios en blanco y símbolos de puntuación, reduciendo el ruido. También pasaré los tokens a minúsculas, y crearé otra aproximación a través de los lemas de los tokens para reducir la dimensionalidad. 

In [50]:
con = sqlite3.connect('../../db/Stackoverflow.db')
cursor = con.cursor()
rows = cursor.execute("Select * from questions").fetchall()
questions = []
answers = []
tags =[]
vocabulary = []
vocabulary_lemma=[]
docs =[]
docs_lemma =[]
classes = []
ids=[]

for row in rows:
    
    id = row[0]
    text = row[1] + row[2]
    doc = nlp(text)#uno pregunta y comentario
    question = []
    question_lemma = []
    for token in doc:
        if token.is_alpha or token.is_digit:#limpio simbolos de puntuación, me quedo con caracteres y números
            
            question.append(token.text.lower())
            question_lemma.append(token.lemma_.lower())
    vocabulary.append(' '.join(question))
    vocabulary_lemma.append(' '.join(question_lemma))

    questions.append(text)
    answers.append(row[4])
    tagsRows = cursor.execute("Select t.tag from questions_tags as qt inner join tags as t on qt.id_tag=t.id where qt.id_question=?",(id,)).fetchall()
    tagList = []
    for tag in tagsRows:
        tagList.append(tag[0])
    classes.extend(tagList)
    tagList=sorted(tagList)
    tags.append(';'.join(tagList))
    docs.append((question,tagList))
    docs_lemma.append((question_lemma,tagList))

con.close()


Creo el vocabulario con las palabras utilizadas en los títulos de las preguntas. Pruebo dos formas, una con la propia palabra y otra con los lemas de las palabras. Vemos que partiendo de los lemas se reduce la dimensionalidad del vector vocabulario.

In [51]:
import pickle
with open('vocabulary_full.pickle', 'wb') as ecn_file:
    pickle.dump(vocabulary, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('vocabulary_lemma_full.pickle', 'wb') as handle:
    pickle.dump(vocabulary_lemma, handle, protocol=pickle.HIGHEST_PROTOCOL)

vocabulary[1], tags[1]

('how do you make this code more you guys please tell me how i can make the following code more pythonic the code is correct full disclosure it problem in handout 4 of this machine learning course i supposed to use newton algorithm on the two data sets for fitting a logistic hypothesis but they use matlab i using scipy eg one question i have is the matrixes kept rounding to integers until i initialized one value to is there a better way thanks',
 'machine-learning;python;scipy')

Lo mismo hago para construir las clases. Aquí utilizo dos aproximaciones, una por concatenado alfabético de tags y otra con los tags por separado. La reducción es considerable si obtengo los términos de forma independiente, diferenciando 967 clases diferentes.

In [52]:
# to save the fitted label
with open('tags_full.pickle', 'wb') as ecn_file:
    pickle.dump(tags, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('classes_full.pickle', 'wb') as handle:
    pickle.dump(classes, handle, protocol=pickle.HIGHEST_PROTOCOL)

classes = list(sorted(set(classes)))
tags =list(sorted(set(tags)))
print(len(tags))
print(len(classes))

# to save the fitted label
with open('tags.pickle', 'wb') as ecn_file:
    pickle.dump(tags, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('classes.pickle', 'wb') as handle:
    pickle.dump(classes, handle, protocol=pickle.HIGHEST_PROTOCOL)


3192
967


Creo un CountVectorizer para crear el vector que codifique la pregunta, conocido como bags of words. Se le pasa el conjunto de tokens de la pregunta, que tendrá que estar tratado de la misma forma que el vocabulario, que se pasará como segundo parámetro.

In [55]:


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
cv = CountVectorizer(binary = True,
                    ngram_range=(1,3))
X_train = cv.fit_transform(vocabulary)


<1x694108 sparse matrix of type '<class 'numpy.int64'>'
	with 141 stored elements in Compressed Sparse Row format>

In [59]:
with open('tags_full.pickle', 'rb') as handle:
    y_tags = pickle.load(handle)

X_train.shape, len(y_tags)

((6839, 694108), 6839)

Creo una función para crear el vector que codifique la pregunta, conocido como bags of words. Se le pasa el conjunto de tokens de la pregunta, que tendrá que estar tratado de la misma forma que el vocabulario, que se pasará como segundo parámetro.

In [ ]:

lbl_encoder = LabelEncoder()

y_test = lbl_encoder.fit_transform(training_labels)

In [122]:
import numpy as np
def getBagWords(tokens,vocabulary):
    bag = []
    for word in vocabulary:
        if word in tokens:
            bag.append(1)
        else:
            bag.append(0)
    return np.array(bag)


bag = getBagWords(docs_lemma[7][0],vocabulary_lemma)
len(bag)

3968

## Preprocesamineto con keras

Keras proporciona funcionalidad para realizar todo el tratamiento anterior de una forma más sencilla.
A pesar de esta facilidad, keras no posee ni lematización ni steaming, por lo que habrá que realizarlo con otra libreria


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
# definir 5 documentos
docs = ['Well done!',
   'Good work',
   'Great effort',
   'nice work',
   'Excellent!']
# crear el tokenizador
t = Tokenizer()
# ajustar el tokenizador en los documentos
t.fit_on_texts(questions)
# resumir lo que se aprendió
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# enteros codificando documentos. el mode puede ser igual a
#binary: Si cada palabra está presente o no en el documento. Este es el valor por defecto.
#count: El recuento de cada palabra del documento.
#tfidf: La puntuación de Frecuencia de documento inversa de texto (TF-IDF) para cada palabra del documento.
#freq: La frecuencia de cada palabra como una relación de palabras dentro de cada documento.
encoded_docs = t.texts_to_matrix(docs, mode='freq')
print(encoded_docs)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])
5
{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8}
defaultdict(<class 'int'>, {'done': 1, 'well': 1, 'work': 2, 'good': 1, 'great': 1, 'effort': 1, 'nice': 1, 'excellent': 1})
[[0.  0.  0.5 0.5 0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.5 0.  0. ]
 [0.  0.5 0.  0.  0.  0.  0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  1. ]]
